In [1]:
import pandas as pd
from datetime import date

list_of_bindingDB = ['source_files/BindingDB_BindingDB_Inhibition_12_12_2019.tsv','source_files/BindingDB_PubChem_12_12_2019.tsv','source_files/BindingDB_USPatent_12_12_2019.tsv']
df_final = pd.DataFrame()
today = date.today()
final_output = 'BindingDB_'+today.strftime("%d_%m_%y")
for file_path in list_of_bindingDB:
    output = file_path.rstrip('.tsv')+'_clean.tsv'
    with open(file_path,'r') as tsv:
        line_list=[]
        counter = 0
        for lines in tsv:
            lines.rstrip('\n')
            splitted = lines.split('\t')
            if counter == 0:
                header_len = len(splitted)
            splitted = splitted[0:header_len]
            to_print = '\t'.join(splitted)      
            counter+=1
            line_list.append(to_print)
    with open(output,'w') as tsv_out:
        for i in line_list:
            tsv_out.write(i)


    header = ['BindingDB Ligand Name', 'Ligand SMILES', 'Ligand InChI',
              'Ligand InChI Key','Target Name Assigned by Curator or DataSource',
              'UniProt (SwissProt) Primary ID of Target Chain',
              'IC50 (nM)', 'Ki (nM)','Kd (nM)', 'EC50 (nM)', 'kon (M-1-s-1)', 'koff (s-1)',
              'pH', 'Temp (C)','Article DOI','Patent Number','Institution','ZINC ID of Ligand',
              'Curation/DataSource']
    df = pd.read_csv(output,delimiter='\t',header=0,usecols=header,quoting=3)
    df_final = df_final.append(df)

df_final.rename(columns={'BindingDB Ligand Name':'ligand_name', 'Ligand SMILES':'ligand_smiles', 'Ligand InChI':'Inchi',
              'Ligand InChI Key':'Inchi_key','Target Name Assigned by Curator or DataSource':'target_name',
              'UniProt (SwissProt) Primary ID of Target Chain':'target_id',
              'IC50 (nM)':'IC50(nM)', 'Ki (nM)':'Ki(nM)','Kd (nM)':'Kd(nM)', 'EC50 (nM)':'EC50(nM)', 'kon (M-1-s-1)':'kon(M-1s-1)', 'koff (s-1)':'koff(s-1)',
              'Temp (C)':'Temp','Article DOI':'DOI','Patent Number':'Patent_number','Institution':'Institution','ZINC ID of Ligand':'ZincID',
              'Curation/DataSource':'Source'},inplace=True)
df_final.reset_index(drop=True,inplace=True)
df_final.to_json('json_files/'+final_output+'.json')


/ssddata/sdecesco/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (10,13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/ssddata/sdecesco/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (8,15,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/ssddata/sdecesco/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (10,11,15,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


FileNotFoundError: [Errno 2] File b'source_files/bindingDB_purchasable_18_02_2019.tsv' does not exist: b'source_files/bindingDB_purchasable_18_02_2019.tsv'

In [2]:

purchasable_file_name = 'source_files/BindingDB_purchasable_12_12_2019.tsv'

purch = pd.read_csv(purchasable_file_name,delimiter='\t',header=0,quoting=3)
purch.drop(columns=['Target Name','bindingDB ID'],inplace=True)
purch.rename(columns={'UniprotID':'target_id', 'smiles string':'smiles',
       'affinity type':'affinity_type', 'affinity value':'affinity_value', 'source':'website', 'Price':'price'},inplace=True)
purch['affinity_unit'] = 'nM'
purch['op'] = '='
purch.loc[purch.affinity_value.str.startswith('<').fillna(False),'op'] = '<'
purch.loc[purch.affinity_value.str.startswith('<').fillna(False),'affinity_value'] =purch.loc[purch.affinity_value.str.startswith('<').fillna(False),'affinity_value'].apply(lambda x: x.lstrip('<'))
purch.to_json('json_files/purchasable_compounds_'+today.strftime("%d_%m_%y")+'.json')